In [ ]:
!unzip etl_filtered_v1.zip

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from numpy import arange
import datetime
import seaborn as sns

In [ ]:
df = pd.read_csv('etl_filtered_v1.csv')
df = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
df

,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_DESCR,STAT_CAUSE_CODE,CONT_DATE,DISCOVERY_DATE,CONT_DOY,DISCOVERY_TIME,CONT_TIME,FIRE_SIZE,...,STATE,datetime,population_density,ELEVATION,AWND,PRCP,TMAX,TMIN,distance,month
0,2011,47,Debris Burning,5.0,2455608.5,2455608.5,47.0,1300.0,1530.0,26.0,...,AL,2011-02-16,7.572142,100.6,8.062714,0.0,71.000000,32.000000,0.141421,2
1,2011,47,Debris Burning,5.0,2455610.5,2455608.5,49.0,820.0,1500.0,22.0,...,AL,2011-02-16,31.442815,201.5,8.048481,0.0,76.942800,49.340830,0.100000,2
2,2011,48,Arson,7.0,2455609.5,2455609.5,48.0,2030.0,2300.0,209.0,...,AL,2011-02-17,7.572142,100.6,8.064342,0.0,70.000000,33.000000,0.200000,2
3,2011,50,Arson,7.0,2455617.5,2455611.5,56.0,1800.0,1500.0,645.0,...,AL,2011-02-19,49.942890,182.9,8.037152,0.0,66.000000,49.000000,0.282843,2
4,2011,51,Debris Burning,5.0,2455625.5,2455612.5,64.0,1500.0,1900.0,231.0,...,AL,2011-02-20,49.942890,189.0,8.042611,0.0,76.657125,49.084371,0.100000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367739,2015,75,Miscellaneous,9.0,NaN,2457097.5,NaN,NaN,NaN,2.1,...,VA,2015-03-16,33.568885,283.5,7.366097,0.0,61.000000,39.000000,0.100000,3
367740,2015,95,Miscellaneous,9.0,NaN,2457117.5,NaN,NaN,NaN,1.0,...,VA,2015-04-05,33.568885,283.5,7.358285,0.0,57.000000,39.000000,0.100000,4
367741,2015,37,Miscellaneous,9.0,NaN,2457059.5,NaN,NaN,NaN,2.0,...,VA,2015-02-06,38.354455,247.8,7.406834,0.0,70.218508,43.916184,0.100000,2
367742,2015,89,Miscellaneous,9.0,NaN,2457111.5,NaN,NaN,NaN,2.0,...,VA,2015-03-30,38.354455,208.2,7.384267,0.0,69.862021,43.603630,0.141421,3


In [ ]:
def get_datetime(row):
  year = row["FIRE_YEAR"]
  doy = row["DISCOVERY_DOY"]
  return datetime.datetime.strptime(f"{year} {doy}", '%Y %j')

def get_month(row):
  year = row["FIRE_YEAR"]
  doy = row["DISCOVERY_DOY"]
  return datetime.datetime.strptime(f"{year} {doy}", '%Y %j').month

In [ ]:
df["month"] = df.apply(lambda row: get_month(row), axis=1)

In [ ]:
df = df[df["STAT_CAUSE_DESCR"] != 'Missing/Undefined']

In [ ]:
df1 = df.groupby(["STATE", "month"])['FIRE_SIZE'].mean().reset_index(name="size")

In [ ]:
df1

,STATE,month,size
0,AK,1,0.100000
1,AK,2,0.100000
2,AK,3,4.550000
3,AK,4,4.395455
4,AK,5,1725.935088
...,...,...,...
595,WY,8,264.300924
596,WY,9,73.956454
597,WY,10,13.480263
598,WY,11,19.404167


In [ ]:
state_df = pd.read_json("allstates.json")

In [ ]:
state_df["val"] = state_df["val"].apply(str)
state_df

In [ ]:
merged_df = df1.merge(state_df, how='inner', left_on='STATE', right_on='id')

In [ ]:
merged_df

,STATE,month,size,id,val
0,AK,1,0.100000,AK,2
1,AK,2,0.100000,AK,2
2,AK,3,4.550000,AK,2
3,AK,4,4.395455,AK,2
4,AK,5,1725.935088,AK,2
...,...,...,...,...,...
595,WY,8,264.300924,WY,56
596,WY,9,73.956454,WY,56
597,WY,10,13.480263,WY,56
598,WY,11,19.404167,WY,56


In [ ]:
merged_df.to_csv("merged.csv", index=False)